In this notebook, we process the raw dataset from the Kaggle Customer Service competition to make it suitable for use with a RAG llm chatbot.

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None
import pickle
from langdetect import detect
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk


In [2]:

tweets = pd.read_csv("twcs/twcs.csv")
tweets['created_at'] = pd.to_datetime(tweets['created_at'])
tweets

/tmp/ipykernel_28596/1865298698.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  tweets['created_at'] = pd.to_datetime(tweets['created_at'])


,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,2017-10-31 22:10:47+00:00,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,2017-10-31 22:11:45+00:00,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,2017-10-31 22:08:27+00:00,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,2017-10-31 21:54:49+00:00,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,2017-10-31 21:49:35+00:00,@sprintcare I did.,4,6.0
...,...,...,...,...,...,...,...
2811769,2987947,sprintcare,False,2017-11-22 08:43:51+00:00,"@823869 Hey, we'd be happy to look into this f...",NaN,2987948.0
2811770,2987948,823869,True,2017-11-22 08:35:16+00:00,@115714 wtf!? I’ve been having really shitty s...,2987947,NaN
2811771,2812240,121673,True,2017-11-23 04:13:07+00:00,@143549 @sprintcare You have to go to https://...,NaN,2812239.0
2811772,2987949,AldiUK,False,2017-11-22 08:31:24+00:00,"@823870 Sounds delicious, Sarah! 😋 https://t.c...",NaN,2987950.0


In [3]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Define a function to clean the text
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove mentions and hashtags
    text = re.sub(r'@\w+|#\w+', '', text)
    
    # Remove special characters and numbers
    text = re.sub(r'\W+|\d+', ' ', text)
    
    # Expand contractions (Here we skip this step due to complexity, but it can be done using libraries like contractions)
    
    # Lemmatize text
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    
    return text

# Apply the cleaning function to the text column
tweets['text'] = tweets['text'].apply(clean_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/shtabari/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/shtabari/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/shtabari/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:

def get_lang_detector(nlp, name):
    return LanguageDetector()
nlp = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)


In [5]:
text = "i pre ordered my game at game stop"
doc = nlp(text)
doc._.language

{'language': 'da', 'score': 0.999995521726347}

In [6]:
def remove_noneng(text):
    doc = nlp(text)
    return text if doc._.language['language'] == 'en' else ''

In [7]:
# tweets['text'] = tweets['text'].apply(remove_noneng)

In [8]:
with open('emoji.pickle', 'rb') as handle:
    EMO_UNICODE = pickle.load(handle)
UNICODE_EMO = {v: k for k, v in EMO_UNICODE.items()}


In [9]:
### remove emojis
def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = re.sub(r'('+emot+')', "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()), text)
    return text

text = "game is on 🔥"
convert_emojis(text)

'game is on fire'

In [10]:
# tweets['text'] = tweets['text'].apply(convert_emojis)

In [11]:
### we join on tweet_id and in_response_to_tweet_id to get conversion in one (or a few) rows
first_inbound = tweets[pd.isnull(tweets.in_response_to_tweet_id) & tweets.inbound]
inbounds_and_outbounds = pd.merge(first_inbound, tweets, left_on='tweet_id', 
                                  right_on='in_response_to_tweet_id')#.sample(frac=100)
inbounds_and_outbounds = inbounds_and_outbounds[inbounds_and_outbounds.inbound_y ^ True]
inbounds_and_outbounds


,tweet_id_x,author_id_x,inbound_x,created_at_x,text_x,response_tweet_id_x,in_response_to_tweet_id_x,tweet_id_y,author_id_y,inbound_y,created_at_y,text_y,response_tweet_id_y,in_response_to_tweet_id_y
0,8,115712,True,2017-10-31 21:45:10+00:00,is the worst customer service,"9,6,10",NaN,6,sprintcare,False,2017-10-31 21:46:24+00:00,can you please send u a private message so tha...,"5,7",8.0
1,8,115712,True,2017-10-31 21:45:10+00:00,is the worst customer service,"9,6,10",NaN,9,sprintcare,False,2017-10-31 21:46:14+00:00,i would love the chance to review the account ...,NaN,8.0
2,8,115712,True,2017-10-31 21:45:10+00:00,is the worst customer service,"9,6,10",NaN,10,sprintcare,False,2017-10-31 21:45:59+00:00,hello we never like our customer to feel like ...,NaN,8.0
3,18,115713,True,2017-10-31 19:56:01+00:00,y all lie about your great connection bar lte ...,17,NaN,17,sprintcare,False,2017-10-31 19:59:13+00:00,h there we d definitely like to work with you ...,16,18.0
4,20,115715,True,2017-10-31 22:03:34+00:00,whenever i contact customer support they tell ...,19,NaN,19,sprintcare,False,2017-10-31 22:10:10+00:00,please send me a private message so that i can...,NaN,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875287,2987942,823867,True,2017-11-22 07:30:39+00:00,hai how many day needed to proceed cancelation...,2987941,NaN,2987941,AirAsiaSupport,False,2017-11-22 07:55:05+00:00,we have replied you via dm thanks emir,NaN,2987942.0
875288,2987944,823868,True,2017-11-22 07:43:36+00:00,i am unable to do web checkin i am getting the...,2987943,NaN,2987943,AirAsiaSupport,False,2017-11-22 07:54:57+00:00,sorry but kindly try to clear browser cache co...,NaN,2987944.0
875289,2987946,524544,True,2017-11-22 08:25:48+00:00,hope you are well doe the am train from crewe ...,2987945,NaN,2987945,VirginTrains,False,2017-11-22 08:27:34+00:00,that s a peak service the is the first off pea...,NaN,2987946.0
875290,2987948,823869,True,2017-11-22 08:35:16+00:00,wtf i ve been having really shitty service all...,2987947,NaN,2987947,sprintcare,False,2017-11-22 08:43:51+00:00,hey we d be happy to look into this for you pl...,NaN,2987948.0


In [12]:
company_info = inbounds_and_outbounds.author_id_y.value_counts().reset_index()
company_info

,author_id_y,count
0,AmazonHelp,84637
1,AppleSupport,74632
2,Uber_Support,40029
3,Delta,28485
4,SpotifyCares,26940
...,...,...
103,mediatemplehelp,190
104,AskDSC,189
105,CarlsJr,138
106,HotelTonightCX,131


In [13]:
### Filter for Amazon and Apple
def comp_filter(x):
    if x.lower().find("amazon") == 0 or x.lower().find("apple") == 0:
        return True
    return False
df_aa = inbounds_and_outbounds[inbounds_and_outbounds.author_id_y.apply(lambda x: comp_filter(x))]
df_aa

,tweet_id_x,author_id_x,inbound_x,created_at_x,text_x,response_tweet_id_x,in_response_to_tweet_id_x,tweet_id_y,author_id_y,inbound_y,created_at_y,text_y,response_tweet_id_y,in_response_to_tweet_id_y
49,272,115770,True,2017-11-22 09:14:39+00:00,amazonのfiretvstickが見れない,269,NaN,269,AmazonHelp,False,2017-11-22 09:23:01+00:00,こんにちは アマゾン公式です fire tv stickが見れないというのは どのような状況...,"270,271",272.0
64,325,115792,True,2017-11-22 08:55:35+00:00,amazonプライムビデオ 再生エラーが多いです,324,NaN,324,AmazonHelp,False,2017-11-22 09:06:00+00:00,ご不便をおかけしております アプリをご利用でしょうか 強制停止 gt 端末の再起動にて改善す...,NaN,325.0
92,617,115820,True,2017-10-31 22:16:32+00:00,way to drop the ball on customer service so pi...,615,NaN,615,AmazonHelp,False,2017-10-31 22:29:00+00:00,i m sorry we ve let you down without providing...,616,617.0
93,621,115822,True,2017-10-31 22:19:34+00:00,i want my amazon payment account closed dm me ...,620,NaN,620,AmazonHelp,False,2017-10-31 22:28:34+00:00,i am unable to affect your account via twitter...,NaN,621.0
94,624,115824,True,2017-10-31 22:12:37+00:00,also beim addams family film in prime sind bil...,622,NaN,622,AmazonHelp,False,2017-10-31 22:28:00+00:00,hi wir erhalten die filme serien so vom jeweil...,623,624.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875241,2987820,823830,True,2017-11-22 06:00:24+00:00,amazonで注文したdvdがまだ届かない 昨日まで到着予定なんだけど問い合わせするかなぁ,2987819,NaN,2987819,AmazonHelp,False,2017-11-22 06:19:27+00:00,ご心配をおかけしております ご案内しているお届け予定日を経過しても未着の場合はカスタマーサー...,NaN,2987820.0
875242,2987822,823831,True,2017-11-22 06:10:09+00:00,たいへん amazonでピンク動画見たら請求来た 私の消息途絶えたらamazonに社会から消...,2987821,NaN,2987821,AmazonHelp,False,2017-11-22 06:17:26+00:00,該当のメッセージは ym,NaN,2987822.0
875270,2987901,823853,True,2017-11-22 07:07:30+00:00,架空請求きたよww しかとショートメールでamazon相談係,2987900,NaN,2987900,AmazonHelp,False,2017-11-22 07:09:54+00:00,ご承知のとおり 残念ながら悪質な詐欺が増加しているようですのでお気を付けください を装った詐...,NaN,2987901.0
875271,2987903,823854,True,2017-11-22 07:04:14+00:00,señores de estoy confuso sobre el público dest...,2987902,NaN,2987902,AmazonHelp,False,2017-11-22 07:09:53+00:00,hola crispín lamento mucho la confusión causad...,NaN,2987903.0


In [14]:
### We just keep english conversations 
df_aa['text_x'] = df_aa['text_x'].apply(remove_noneng)
df_aa['text_y'] = df_aa['text_y'].apply(remove_noneng)

In [16]:
### drop empty and null vals
df_aa['text_x'].replace('', np.nan, inplace=True)#.isnull().sum()
df_aa['text_y'].replace('', np.nan, inplace=True)
df_aa.dropna(axis=0,how='any',subset=['text_x','text_y'],inplace=True)

In [18]:
### manually inspect the data and get rid of rows that escape from filters
drop_rows = [268666,268667,229174,184589,170714,21934,165742]
df_aa.drop(axis=0,index=drop_rows,inplace=True)

In [19]:
df_aa.to_pickle('rag_data.pkl')

In [20]:
df_aa

,tweet_id_x,author_id_x,inbound_x,created_at_x,text_x,response_tweet_id_x,in_response_to_tweet_id_x,tweet_id_y,author_id_y,inbound_y,created_at_y,text_y,response_tweet_id_y,in_response_to_tweet_id_y
92,617,115820,True,2017-10-31 22:16:32+00:00,way to drop the ball on customer service so pi...,615,NaN,615,AmazonHelp,False,2017-10-31 22:29:00+00:00,i m sorry we ve let you down without providing...,616,617.0
93,621,115822,True,2017-10-31 22:19:34+00:00,i want my amazon payment account closed dm me ...,620,NaN,620,AmazonHelp,False,2017-10-31 22:28:34+00:00,i am unable to affect your account via twitter...,NaN,621.0
96,632,115829,True,2017-10-31 21:34:58+00:00,my package wa accidentally opened item missing...,631,NaN,631,AmazonHelp,False,2017-10-31 22:27:00+00:00,i m sorry your order arrived in this condition...,NaN,632.0
97,634,115831,True,2017-10-31 21:39:58+00:00,why is my order at my local courier for the la...,633,NaN,633,AmazonHelp,False,2017-10-31 22:26:37+00:00,i m sorry for the wait please reach out to u s...,NaN,634.0
98,636,115832,True,2017-10-31 22:05:36+00:00,thanks for the style advice look i think,635,NaN,635,AmazonHelp,False,2017-10-31 22:26:07+00:00,alexa say both style are working for you my vo...,NaN,636.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875171,2987663,823779,True,2017-11-22 03:24:02+00:00,what the fuck my phone keep hanging up call an...,2987662,NaN,2987662,AppleSupport,False,2017-11-22 03:27:30+00:00,we d like to help send u a dm and we can start...,NaN,2987663.0
875176,2987675,823783,True,2017-11-22 03:29:16+00:00,got my at am thanks for fulfilling the order fast,2987673,NaN,2987673,AmazonHelp,False,2017-11-22 03:49:29+00:00,woohoo that s awesome hope you love the phone j,2987674,2987675.0
875199,2987723,823796,True,2017-11-22 04:07:23+00:00,is anyone having issue with there phone freezi...,2987722,NaN,2987722,AppleSupport,False,2017-11-22 04:15:29+00:00,we d love to offer our help in making sure you...,NaN,2987723.0
875205,2987736,823802,True,2017-11-22 04:48:51+00:00,no exchange available for i need to exchange m...,2987735,NaN,2987735,AmazonHelp,False,2017-11-22 05:22:31+00:00,the exchange offer is currently available only...,NaN,2987736.0


In [21]:
### order the data by author and date
df_aa = df_aa.sort_values(by=['author_id_x','author_id_y','created_at_x'])
df_aa

,tweet_id_x,author_id_x,inbound_x,created_at_x,text_x,response_tweet_id_x,in_response_to_tweet_id_x,tweet_id_y,author_id_y,inbound_y,created_at_y,text_y,response_tweet_id_y,in_response_to_tweet_id_y
504305,1804419,10423,True,2017-10-13 17:08:39+00:00,every io update ha slowed my phone down gyst,1804417,NaN,1804417,AppleSupport,False,2017-10-13 17:34:13+00:00,we re happy to help you with this can you tell...,1804418,1804419.0
407388,1477605,10766,True,2017-11-02 21:07:32+00:00,weird the rubber cable on two of my macbook ai...,1477604,NaN,1477604,AppleSupport,False,2017-11-02 23:04:56+00:00,we can totally help you look at some option fo...,NaN,1477605.0
151360,569292,10766,True,2017-12-03 02:38:35+00:00,friend in australia and any idea where i can b...,569291,NaN,569291,AppleSupport,False,2017-12-03 02:44:30+00:00,reach out to our apple online store team via t...,NaN,569292.0
371939,1352916,115724,True,2017-10-27 15:57:51+00:00,if you like your iphone gt do not download io ...,1352915,NaN,1352915,AppleSupport,False,2017-10-27 18:24:01+00:00,we d like to help if you re having any issue w...,NaN,1352916.0
484788,1743637,115773,True,2017-11-07 05:33:22+00:00,tell me why apple music deleted lemonade off m...,"1743638,1743635",NaN,1743635,AppleSupport,False,2017-11-07 14:31:27+00:00,hi we are here to help have you attempted to r...,1743636,1743637.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875199,2987723,823796,True,2017-11-22 04:07:23+00:00,is anyone having issue with there phone freezi...,2987722,NaN,2987722,AppleSupport,False,2017-11-22 04:15:29+00:00,we d love to offer our help in making sure you...,NaN,2987723.0
875205,2987736,823802,True,2017-11-22 04:48:51+00:00,no exchange available for i need to exchange m...,2987735,NaN,2987735,AmazonHelp,False,2017-11-22 05:22:31+00:00,the exchange offer is currently available only...,NaN,2987736.0
875240,2987818,823829,True,2017-11-22 05:51:23+00:00,there should be bonus and gift for regular cus...,2987816,NaN,2987816,AmazonHelp,False,2017-11-22 06:20:00+00:00,we do not have any such offer at this point in...,2987817,2987818.0
185556,687902,83673,True,2017-11-23 07:03:12+00:00,i restored a backup from itunes onto my new ip...,687900,NaN,687900,AppleSupport,False,2017-11-23 14:27:30+00:00,we have plenty of other step we can take here ...,687901,687902.0


In [22]:
### combining columns and create prompts for LLM
def make_conversion(row):
    question = row['text_x']
    answer = row['text_y']
    return "Customer: " + question + ". Customer Service: " + answer

In [23]:
df_final = df_aa.apply(lambda row: make_conversion(row),axis = 1).reset_index()
df_final

,index,0
0,504305,Customer: every io update ha slowed my phone d...
1,407388,Customer: weird the rubber cable on two of my ...
2,151360,Customer: friend in australia and any idea whe...
3,371939,Customer: if you like your iphone gt do not do...
4,484788,Customer: tell me why apple music deleted lemo...
...,...,...
134152,875199,Customer: is anyone having issue with there ph...
134153,875205,Customer: no exchange available for i need to ...
134154,875240,Customer: there should be bonus and gift for r...
134155,185556,Customer: i restored a backup from itunes onto...


In [28]:
### Save the as text file 
np_arr = df_final[0].values
np.savetxt('rag_data.txt',np_arr,fmt='%s')